In [1]:
import pysftp
from settings import ftp_server, ftp_port, ftp_user, ftp_password

import pandas as pd
import os

In [2]:
#
# setup output directory
# 
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'

In [3]:
# open the connection
try:
    conn = pysftp.Connection(host=ftp_server,port=ftp_port,username=ftp_user, password=ftp_password)
    print("connection established successfully")
except:
    print('failed to establish connection to targeted server')

connection established successfully


In [4]:
# change to project directory
remote_directory = '/bil/proj/u19zeng'
conn.cwd(remote_directory)

# change to downsampled directory
conn.cwd('fMOST_downsampled/brain')

In [5]:
# list directory structure
directory_structure = conn.listdir_attr()

In [6]:
# close the connection
conn.close()

In [7]:
# create a data frame and parse downsampled filename
dsdf = pd.DataFrame([attr.filename for attr in directory_structure], columns = ['Filename'])

In [8]:
dsdf

,Filename
0,15257red_xy32z8.tif
1,16130red_xy32z8.tif
2,17051red_xy32z8.tif
3,17107red_xy32z8.tif
4,17109green_xy32z8.nrrd
...,...
166,mouse194074green_xy32z8.tif
167,mouse194075green_xy32z8.tif
168,mouse194083green_xy32z8.tif
169,mouse227845green_xy32z8.tif


In [9]:
# process filename
flist = dsdf['Filename'].values

In [10]:
# find and strip extension
dsdf["Image Type"] = [ os.path.splitext(f)[1] for f in flist ] 
flist = [ os.path.splitext(f)[0] for f in flist ] 

In [11]:
# process num bits
dsdf["Number of Bits"] = None

idx = dsdf['Filename'].str.contains('8bit')
dsdf.loc[idx,"Number of Bits"] = 8

idx = dsdf['Filename'].str.contains('16bit')
dsdf.loc[idx,"Number of Bits"] = 16

flist = [f.replace('8bit','') for f in flist]
flist = [f.replace('16bit','') for f in flist]

In [12]:
# process channel
dsdf["Channel"] = None

idx = dsdf['Filename'].str.contains('red')
dsdf.loc[idx,"Channel"] = 'red'

idx = dsdf['Filename'].str.contains('green')
dsdf.loc[idx,"Channel"] = 'green'

flist = [f.replace('red','') for f in flist]
flist = [f.replace('green','') for f in flist]

In [13]:
# process specimen id
dsdf["HUST Sample ID"] = None

flist = [f.replace('mouse','') for f in flist]
dsdf["HUST Sample ID"] = [f.split('_')[0] for f in flist]
flist = [f + '_' for f in flist]
flist = [f.split('_')[1] for f in flist]

In [14]:
# process resolution
flist = [f.replace('xy','') for f in flist]
flist = [f.replace('z','_') for f in flist]
flist = [f.replace('rot','_') for f in flist]
dsdf["temp"] = flist

dsdf['XY Downsample'] = None
dsdf['Z Downsample'] = None
dsdf['Rot'] = None

for index, row in dsdf.iterrows() :
    
    if '_' in row['temp'] :
        
        rlist = row['temp'].split('_')
        
        if len(rlist) > 0 :
            dsdf.at[index,'XY Downsample'] = int(rlist[0])
            
        if len(rlist) > 1 :
            dsdf.at[index,'Z Downsample'] = int(rlist[1])
            
        if len(rlist) > 2 :
            dsdf.at[index,'Rot'] = int(rlist[2])
            
dsdf.drop(['temp'], axis=1, inplace=True)

In [15]:
dsdf

,Filename,Image Type,Number of Bits,Channel,HUST Sample ID,XY Downsample,Z Downsample,Rot
0,15257red_xy32z8.tif,.tif,None,red,15257,32,8,None
1,16130red_xy32z8.tif,.tif,None,red,16130,32,8,None
2,17051red_xy32z8.tif,.tif,None,red,17051,32,8,None
3,17107red_xy32z8.tif,.tif,None,red,17107,32,8,None
4,17109green_xy32z8.nrrd,.nrrd,None,green,17109,32,8,None
...,...,...,...,...,...,...,...,...
166,mouse194074green_xy32z8.tif,.tif,None,green,194074,32,8,None
167,mouse194075green_xy32z8.tif,.tif,None,green,194075,32,8,None
168,mouse194083green_xy32z8.tif,.tif,None,green,194083,32,8,None
169,mouse227845green_xy32z8.tif,.tif,None,green,227845,32,8,None


In [16]:
#
# write out data frame
#
output_path = os.path.join(output_directory,'downsampled_images_inventory.csv')
dsdf.to_csv( output_path, index=False )